# Figure 3 amino acid mutation matrix and collapsing of clade phylogenetic tree

This notebook was used to 

1. parse Nextclade amino acid (AA) mutation information into an AA matrix compatible for visualization with [ggtree gheatmap](https://github.com/YuLab-SMU/ggtree/blob/master/R/gheatmap.R) with some code to check if sequence info was missing for certain mutations (i.e. Ns or gaps in the location of a mutation). Only the AA mutations present in the ON WTD sequences were considered and mutations in other sequences were dropped.
2. determine AA mutation patterns of 157 taxa highlighted in Figure 2 global tree
3. collapse/prune nodes with duplicated AA mutation patterns relative to ON WTD sequences
4. output a tree with collapsed taxa pruned away and basic table with counts of number of collapsed taxa


In [32]:
# read Nextclade GFF to get coordinates of genes
df_gff = pd.read_table('nextlade-data/sars-cov-2/genemap.gff', comment='#', header=None)
df_gff

,0,1,2,3,4,5,6,7,8
0,.,.,gene,26245,26472,.,+,.,gene_name=E
1,.,.,gene,26523,27191,.,+,.,gene_name=M
2,.,.,gene,28274,29533,.,+,.,gene_name=N
3,.,.,gene,266,13468,.,+,.,gene_name=ORF1a
4,.,.,gene,13468,21555,.,+,.,gene_name=ORF1b
5,.,.,gene,25393,26220,.,+,.,gene_name=ORF3a
6,.,.,gene,27202,27387,.,+,.,gene_name=ORF6
7,.,.,gene,27394,27759,.,+,.,gene_name=ORF7a
8,.,.,gene,27756,27887,.,+,.,gene_name=ORF7b
9,.,.,gene,27894,28259,.,+,.,gene_name=ORF8


In [33]:
df_gff = df_gff.loc[:,[3,4,8]]
df_gff.columns = ['start', 'end', 'gene']

In [50]:
df_gff.gene = df_gff.gene.str.replace('gene_name=', '').str.strip()

In [51]:
df_gff

,start,end,gene
0,26245,26472,E
1,26523,27191,M
2,28274,29533,N
3,266,13468,ORF1a
4,13468,21555,ORF1b
5,25393,26220,ORF3a
6,27202,27387,ORF6
7,27394,27759,ORF7a
8,27756,27887,ORF7b
9,27894,28259,ORF8


In [52]:
gff = {row.gene: (row.start, row.end) for row in df_gff.itertuples()}

In [53]:
gff

{'E': (26245, 26472),
 'M': (26523, 27191),
 'N': (28274, 29533),
 'ORF1a': (266, 13468),
 'ORF1b': (13468, 21555),
 'ORF3a': (25393, 26220),
 'ORF6': (27202, 27387),
 'ORF7a': (27394, 27759),
 'ORF7b': (27756, 27887),
 'ORF8': (27894, 28259),
 'ORF9b': (28284, 28577),
 'S': (21563, 25384)}

In [38]:
import re

In [43]:
regex_gene_aa_pos_simple = re.compile(r'(\w+):[a-zA-Z]+(\d+).*')

In [45]:
regex_gene_aa_pos_insertion = re.compile(r'(\w+):(\d+).*')

In [46]:
mut = 'ORF1a:2037:MRAS'
m = regex_gene_aa_pos_simple.match(mut)
if m:
    print(m.groups())
else:
    m = regex_gene_aa_pos_insertion.match(mut)
    if m:
        print(m.groups())

('ORF1a', '2037')


In [47]:
m = regex_gene_aa_pos_simple.match('ORF1a:N2038D')
if m:
    print(m.groups())

('ORF1a', '2038')


# Functions to get the nt position from AA mutation

From `{gene}:*{aa_position}*` to reference nucleotide position range for AA position.

For example `ORF1a:V23D` will map to a nucleotide from 332 to 335 (actual nt change T333A).

The `get_gene_nt_coords` will not work for frameshift mutations, however, but there's only one in the results.

In [62]:
from typing import Dict, Tuple, List

def split_mut(aamut: str) -> (str, int):
    m = regex_gene_aa_pos_simple.match(aamut)
    if m:
        gene, aa_pos = m.groups()
        return gene, int(aa_pos)
    else:
        m = regex_gene_aa_pos_insertion.match(aamut)
        if m:
            gene, aa_pos = m.groups()
            return gene, int(aa_pos)
        else:
            raise ValueError(f'Could not parse gene and AA position from {aamut}')

def get_gene_nt_coords(gff: Dict[str, Tuple[int, int]], gene: str, aa_pos: int) -> (int, int):
    if gene not in gff:
        raise ValueError(f'No gene {gene} in gene coordinate dict: {gff}')
    start, end = gff.get(gene)
    end_nt = aa_pos * 3 + start
    start_nt = end_nt - 3
    return start_nt, end_nt
    

In [65]:
aa_nt_muts = '''
ORF1a:V23D (T333A)
ORF1a:E159A (A741C)
ORF1a:T265I (C1059T)
ORF1a:M297V (A1154G)
ORF1a:H325Q (T1240A)
ORF1a:T619S (C2121G)
ORF1a:T708I (C2388T)
ORF1a:A735 (T2463TA [FRAMESHIFT])
ORF1a:D1289 (GA4130G [FRAMESHIFT])
ORF1a:V1290A (T4134C)
ORF1a:A1314V (C4206T)
ORF1a:A1809 (G5690GA [FRAMESHIFT])
ORF1a:L3116F (C9611T)
ORF1a:S3149F (CC9711TT)
ORF1a:K3353R (A10323G)
ORF1a:D3972V (A12180T)
ORF1a:V3976 (G12188GTT [FRAMESHIFT])
ORF1a:S3983F (C12213T)
ORF1a:C4326R (T13241C)
ORF1a:M4390T (T13434C)
ORF1b:R524C (C15037T)
ORF1b:V1271L (G17278T)
ORF1b:M1693I (G18546T)
ORF1b:P1727S (C18646T)
ORF1b:I2303V (A20374G)
ORF1b:A2469del (GTGC20870G [disruptive_inframe_deletion])
ORF1b:K2579R (A21203G)
S:F486L (T23020G)
S:N501T (A23064C)
S:D614G (A23403G)
S:S640F (C23481T)
S:S1003 (CA24566C [FRAMESHIFT])
S:M1237 (GTA25269G [FRAMESHIFT])
ORF3a:T12I (C25427T)
ORF3a:L219V (T26047G)
E:P71S (C26455T)
ORF8:D35Y (G27996T)
ORF8:E106* (G28209T)
N:P168S (C28775T)
N:S206P (T28889C)
N:T391I (C29445T)
ORF1a:S3983F (C12213T)
S:L1004S (T24573C)
S:A1070 (G24770GA [FRAMESHIFT])
S:A1070E (C24771A)
S:Q1071K (C24773A)
S:E1072K (G24776A)
E:P71S (C26455T)
ORF8:E106* (G28209T)
N:P168S (C28775T)
ORF1a:T265I (C1059T)
ORF1a:Y369 (TA1370T [FRAMESHIFT])
ORF1a:T708I (C2388T)
'''.strip().split('\n')

In [66]:
for aant_mut in aa_nt_muts:
    if ':' not in aant_mut:
        continue
    print(aant_mut)
    aamut, ntmut = aant_mut.split(' ', maxsplit=1)
    gene, aapos = split_mut(aamut)
    print(gene, aapos)
    print(get_gene_nt_coords(gff, gene, aapos))

ORF1a:V23D (T333A)
ORF1a 23
(332, 335)
ORF1a:E159A (A741C)
ORF1a 159
(740, 743)
ORF1a:T265I (C1059T)
ORF1a 265
(1058, 1061)
ORF1a:M297V (A1154G)
ORF1a 297
(1154, 1157)
ORF1a:H325Q (T1240A)
ORF1a 325
(1238, 1241)
ORF1a:T619S (C2121G)
ORF1a 619
(2120, 2123)
ORF1a:T708I (C2388T)
ORF1a 708
(2387, 2390)
ORF1a:A735 (T2463TA [FRAMESHIFT])
ORF1a 735
(2468, 2471)
ORF1a:D1289 (GA4130G [FRAMESHIFT])
ORF1a 1289
(4130, 4133)
ORF1a:V1290A (T4134C)
ORF1a 1290
(4133, 4136)
ORF1a:A1314V (C4206T)
ORF1a 1314
(4205, 4208)
ORF1a:A1809 (G5690GA [FRAMESHIFT])
ORF1a 1809
(5690, 5693)
ORF1a:L3116F (C9611T)
ORF1a 3116
(9611, 9614)
ORF1a:S3149F (CC9711TT)
ORF1a 3149
(9710, 9713)
ORF1a:K3353R (A10323G)
ORF1a 3353
(10322, 10325)
ORF1a:D3972V (A12180T)
ORF1a 3972
(12179, 12182)
ORF1a:V3976 (G12188GTT [FRAMESHIFT])
ORF1a 3976
(12191, 12194)
ORF1a:S3983F (C12213T)
ORF1a 3983
(12212, 12215)
ORF1a:C4326R (T13241C)
ORF1a 4326
(13241, 13244)
ORF1a:M4390T (T13434C)
ORF1a 4390
(13433, 13436)
ORF1b:R524C (C15037T)
ORF1b 524

In [67]:
df_nextclade = pd.read_table('nextclade-output/nextclade.tsv', dtype=str, index_col=0)

In [73]:
df_nextclade.columns

Index(['clade', 'qc.overallScore', 'qc.overallStatus', 'totalSubstitutions',
       'totalDeletions', 'totalInsertions', 'totalFrameShifts',
       'totalAminoacidSubstitutions', 'totalAminoacidDeletions',
       'totalAminoacidInsertions', 'totalMissing', 'totalNonACGTNs',
       'totalPcrPrimerChanges', 'substitutions', 'deletions', 'insertions',
       'privateNucMutations.reversionSubstitutions',
       'privateNucMutations.labeledSubstitutions',
       'privateNucMutations.unlabeledSubstitutions',
       'privateNucMutations.totalReversionSubstitutions',
       'privateNucMutations.totalLabeledSubstitutions',
       'privateNucMutations.totalUnlabeledSubstitutions',
       'privateNucMutations.totalPrivateSubstitutions', 'frameShifts',
       'aaSubstitutions', 'aaDeletions', 'aaInsertions', 'missing',
       'nonACGTNs', 'pcrPrimerChanges', 'alignmentScore', 'alignmentStart',
       'alignmentEnd', 'qc.missingData.missingDataThreshold',
       'qc.missingData.score', 'qc.missingD

In [76]:
missing_series = df_nextclade.missing.str.split(',')

In [87]:
def range_str_to_int_tuple(nt_range: str) -> (int, int):
    if '-' in nt_range:
        start, end = nt_range.split('-')
        return int(start), int(end)
    else:
        return int(nt_range), int(nt_range)

In [79]:
from collections import defaultdict

In [81]:
import numpy as np

In [89]:
sample_missing_regions = {}
for sample, range_missing in missing_series.iteritems():
    if not isinstance(range_missing, list):
        print(f'sample {sample} has no missing regions')
        continue
    sample_missing_regions[sample] = [range_str_to_int_tuple(r) for r in range_missing]

sample mink/USA/MI-CDC-3886940-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886954-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886613-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886955-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886772-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886953-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886875-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886765-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886398-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886391-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886786-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886755-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886793-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886800-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886759-001/2020 has no missing regions
sample mink/USA/MI-CDC-3886854-001/2020 has no missing 

In [90]:
missing_ranges =  sample_missing_regions['4538']
for gmut in ordered_gmuts:
    gene, aapos = split_mut(gmut)
    start, end = get_gene_nt_coords(gff, gene, aapos)
    for missing_start, missing_end in missing_ranges:
        if missing_end >= start >= missing_start or missing_end >= end >= missing_start:
            print(gmut)
            print(start, end)
            print(missing_start, missing_end)
            print('='*80)
    


ORF1a:T708I
2387 2390
2316 2418
ORF1a:A1204V
3875 3878
3770 4018
ORF1a:2037:MRAS
6374 6377
6248 6818
ORF1a:N2038D
6377 6380
6248 6818
ORF1a:T2093I
6542 6545
6248 6818
ORF1a:S2255F
7028 7031
7002 7084
S:V143-
21989 21992
21905 22061
S:Y144-
21992 21995
21905 22061
S:Y145D
21995 21998
21905 22061
S:L1265I
25355 25358
25083 25706
ORF3a:S40L
25510 25513
25083 25706
ORF3a:Q57H
25561 25564
25083 25706
ORF3a:V90F
25660 25663
25083 25706
ORF3a:H93Y
25669 25672
25083 25706


In [1]:
import logging
from pathlib import Path
from typing import Set, List, Dict

import numpy as np
import pandas as pd
import typer


def parse(nextclade_csv: Path) -> pd.DataFrame:
    df_nextclade = pd.read_table(nextclade_csv, sep=';', dtype=str, index_col=0)
    aa_subs: pd.Series = df_nextclade['aaSubstitutions'].str.split(',')
    aa_dels: pd.Series = df_nextclade['aaDeletions'].str.split(',')
    aa_ins: pd.Series = df_nextclade['aaInsertions'].str.split(',')
    nt_subs: pd.Series = df_nextclade['substitutions'].str.split(',')
    sample_aas = sample_to_aa_mutations(aa_subs, aa_dels, aa_ins)
    sample_subs = sample_to_nt_mutations(nt_subs)
    samples = list(sample_aas.keys())
    unique_aas = get_sorted_aa_mutations(sample_aas)
    unique_subs = get_sorted_nt_subs(sample_subs)
    arr_aas = fill_aa_mutation_matrix(sample_aas, samples, unique_aas)
    arr_subs = fill_aa_mutation_matrix(sample_subs, samples, unique_subs)
    dfaa = pd.DataFrame(arr_aas, index=samples, columns=unique_aas)
    dfsubs = pd.DataFrame(arr_subs, index=samples, columns=unique_subs)
    return dfaa, dfsubs

def sample_to_nt_mutations(
        nt_subs: pd.Series
) -> Dict[str, Set[str]]:
    sample_subs = {}
    for sample, nt_sub in zip(nt_subs.index, nt_subs):
        subs = [] if not isinstance(nt_sub, list) else nt_sub
        sample_subs[sample] = set(subs)
    return sample_subs


def sample_to_aa_mutations(
        aa_subs: pd.Series,
        aa_dels: pd.Series
) -> Dict[str, Set[str]]:
    sample_aas = {}
    for sample, aa_sub, aa_del in zip(aa_subs.index, aa_subs, aa_dels):
        aas = [] if not isinstance(aa_sub, list) else aa_sub
        aad = [] if not isinstance(aa_del, list) else aa_del
        sample_aas[sample] = set(aas) | set(aad)
    return sample_aas


def fill_aa_mutation_matrix(
        sample_aas: Dict[str, Set[str]],
        samples: List[str],
        unique_aas: List[str]
) -> np.ndarray:
    """Fill AA mutation matrix with 1 when AA mutation present in sample"""
    arr_aas = np.zeros((len(sample_aas), len(unique_aas)), dtype='uint8')
    for i, sample in enumerate(samples):
        aas = sample_aas[sample]
        for j, aa in enumerate(unique_aas):
            if aa in aas:
                arr_aas[i, j] = 1
    return arr_aas


def get_sorted_aa_mutations(sample_aas: Dict[str, Set[str]]) -> List[str]:
    unique_aas = set()
    for aas in sample_aas.values():
        unique_aas |= aas
    unique_aas = list(unique_aas)
    unique_aas.sort()
    return unique_aas


def get_sorted_nt_subs(sample_subs: Dict[str, Set[str]]) -> List[str]:
    out = set()
    for subs in sample_subs.values():
        out |= subs
    out = list(out)
    out.sort()
    return out


def sample_to_aa_mutations(
        aa_subs: pd.Series,
        aa_dels: pd.Series,
        aa_insertions: pd.Series,
) -> Dict[str, Set[str]]:
    sample_aas = {}
    for sample, aa_sub, aa_del, aa_ins in zip(aa_subs.index, aa_subs, aa_dels, aa_insertions):
        aas = aa_sub if isinstance(aa_sub, list) else []
        aad = aa_del if isinstance(aa_del, list) else []
        aai = aa_ins if isinstance(aa_ins, list) else []
        sample_aas[sample] = set(aas) | set(aad) | set(aai)
    return sample_aas


In [2]:
dfaa, dfsubs = parse('nextclade-output/nextclade.csv')

In [3]:
with open('subtree-samples.txt') as f:
    subtree_names = [l.strip() for l in f if l]

In [4]:
len(subtree_names)

157

In [5]:
dfaa = dfaa.loc[subtree_names,:]

In [6]:
dfaa.shape

(157, 152)

In [7]:
(dfaa.sum(axis=0) == 0)

E:C44Y     False
E:L19F     False
E:P71S     False
M:H125Y    False
N:A119S    False
           ...  
S:V143-    False
S:V70-     False
S:V705A    False
S:Y144-    False
S:Y145D    False
Length: 152, dtype: bool

In [8]:
on_wtd_names = dfaa.index[dfaa.index.str.match(r'^(4\d\d\d|Canada)')]

In [9]:
on_wtd_names

Index(['4538', '4645', '4649', '4534', '4581', '4662', '4658',
       'Canada/ON-PHL-21-44225/2021'],
      dtype='object')

In [10]:
dfaa_on = dfaa.loc[on_wtd_names, :]

In [11]:
mut_mask = dfaa_on.sum() > 0

In [12]:
dfaa_sub = dfaa.loc[:, mut_mask]

In [13]:
dfaa_sub.index.name = 'sample'

In [14]:
ordered_genes = '''
ORF1a
ORF1b
S
ORF3a
E
M
ORF7a
ORF7b
ORF8
ORF9b
N
'''.strip().split()

In [16]:
import re

In [17]:
re.sub(r'(\D+)?(\d+)\D+', r'\2', 'O:123:wha')

'123'

In [18]:
re.sub(r'(\D+)?(\d+)\D+', r'\2', 'O:A123T')

'123'

In [19]:
import re
from collections import defaultdict

def gene_ordered_mutations(muts) -> Dict[str, List[str]]:
    out = defaultdict(list)
    for m in muts:
        gene, mut = m.split(':', maxsplit=1)
        out[gene].append(mut)
    for k, v in out.items():
        v.sort(key=lambda x: int(re.sub(r'(\D+)?(\d+)\D+', r'\2', x)))
    return out

In [20]:
dfaa_sub.columns

Index(['E:L19F', 'E:P71S', 'N:A119S', 'N:P168S', 'N:S206P', 'N:T391I',
       'ORF1a:2037:MRAS', 'ORF1a:A1204V', 'ORF1a:A1283V', 'ORF1a:A1314V',
       'ORF1a:C2210F', 'ORF1a:D4200Y', 'ORF1a:K3353R', 'ORF1a:L2304S',
       'ORF1a:L3116F', 'ORF1a:L3829F', 'ORF1a:L4111F', 'ORF1a:N2038D',
       'ORF1a:S2255F', 'ORF1a:S2500F', 'ORF1a:S3149F', 'ORF1a:S3983F',
       'ORF1a:S4286N', 'ORF1a:T2093I', 'ORF1a:T265I', 'ORF1a:T3398I',
       'ORF1a:T4164I', 'ORF1a:T4174I', 'ORF1a:T708I', 'ORF1a:V2453I',
       'ORF1a:V3718F', 'ORF1b:D1130G', 'ORF1b:D2142Y', 'ORF1b:F685Y',
       'ORF1b:I192V', 'ORF1b:K2579R', 'ORF1b:K2660R', 'ORF1b:L2560F',
       'ORF1b:M1693I', 'ORF1b:P1727S', 'ORF1b:P314L', 'ORF1b:T1453I',
       'ORF1b:T1637I', 'ORF1b:V1271L', 'ORF1b:V364L', 'ORF3a:H93Y',
       'ORF3a:L219V', 'ORF3a:Q57H', 'ORF3a:S40L', 'ORF3a:T221S', 'ORF3a:V90F',
       'ORF8:D35Y', 'ORF8:E106*', 'ORF8:Q27*', 'ORF8:S43F', 'S:D614G',
       'S:F486L', 'S:H49Y', 'S:L1265I', 'S:N501T', 'S:Q613H', 'S:S247G',
 

In [21]:
gmuts = gene_ordered_mutations(dfaa_sub.columns)

In [22]:
ordered_gmuts = [f'{g}:{x}' for g in ordered_genes for x in gmuts[g]]

In [23]:
ordered_gmuts

['ORF1a:T265I',
 'ORF1a:T708I',
 'ORF1a:A1204V',
 'ORF1a:A1283V',
 'ORF1a:A1314V',
 'ORF1a:2037:MRAS',
 'ORF1a:N2038D',
 'ORF1a:T2093I',
 'ORF1a:C2210F',
 'ORF1a:S2255F',
 'ORF1a:L2304S',
 'ORF1a:V2453I',
 'ORF1a:S2500F',
 'ORF1a:L3116F',
 'ORF1a:S3149F',
 'ORF1a:K3353R',
 'ORF1a:T3398I',
 'ORF1a:V3718F',
 'ORF1a:L3829F',
 'ORF1a:S3983F',
 'ORF1a:L4111F',
 'ORF1a:T4164I',
 'ORF1a:T4174I',
 'ORF1a:D4200Y',
 'ORF1a:S4286N',
 'ORF1b:I192V',
 'ORF1b:P314L',
 'ORF1b:V364L',
 'ORF1b:F685Y',
 'ORF1b:D1130G',
 'ORF1b:V1271L',
 'ORF1b:T1453I',
 'ORF1b:T1637I',
 'ORF1b:M1693I',
 'ORF1b:P1727S',
 'ORF1b:D2142Y',
 'ORF1b:L2560F',
 'ORF1b:K2579R',
 'ORF1b:K2660R',
 'S:T22I',
 'S:H49Y',
 'S:T95I',
 'S:V143-',
 'S:Y144-',
 'S:Y145D',
 'S:S247G',
 'S:F486L',
 'S:N501T',
 'S:Q613H',
 'S:D614G',
 'S:V705A',
 'S:L1265I',
 'ORF3a:S40L',
 'ORF3a:Q57H',
 'ORF3a:V90F',
 'ORF3a:H93Y',
 'ORF3a:L219V',
 'ORF3a:T221S',
 'E:L19F',
 'E:P71S',
 'ORF8:Q27*',
 'ORF8:D35Y',
 'ORF8:S43F',
 'ORF8:E106*',
 'N:A119S',
 'N

In [24]:
dfaa_sub[ordered_gmuts]

,ORF1a:T265I,ORF1a:T708I,ORF1a:A1204V,ORF1a:A1283V,ORF1a:A1314V,ORF1a:2037:MRAS,ORF1a:N2038D,ORF1a:T2093I,ORF1a:C2210F,ORF1a:S2255F,...,E:L19F,E:P71S,ORF8:Q27*,ORF8:D35Y,ORF8:S43F,ORF8:E106*,N:A119S,N:P168S,N:S206P,N:T391I
sample,,,,,,,,,,,,,,,,,,,,,
4538,1,0,0,1,1,0,0,0,1,0,...,0,1,0,1,1,1,0,1,1,1
4645,1,1,0,1,1,0,0,0,1,1,...,1,1,0,1,1,1,0,1,1,1
4649,1,1,0,1,1,0,0,0,0,1,...,0,1,0,1,0,1,0,1,1,1
4534,1,1,0,1,1,1,1,1,1,1,...,0,1,0,1,0,1,0,1,1,1
4581,1,1,1,1,1,1,1,0,1,1,...,0,1,0,1,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USA/TX-HMH-MCoV-41555/2020,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USA/TX-HMH-MCoV-8792/2020,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USA/TX-HMH-MCoV-34510/2020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Matrix of AA mutation binary absence/presence to gene name if present, dash if absent

AA mutation sites with no sequencing coverage (Ns or gaps) were filled in with `*No Coverage`

In [25]:
for col in dfaa_sub:
    gene,_ = col.split(':', maxsplit=1)
    series = dfaa_sub[col]
    series[series == 0] = '-'
    series[series == 1] = gene

/home/pkruczkiewicz/miniconda3/lib/python3.9/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)
<ipython-input-25-fcba06cd9054>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series[series == 1] = gene
<ipython-input-25-fcba06cd9054>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series[series == 0] = '-'
/home/pkruczkiewicz/miniconda3/lib/python3.9/site-pac

In [135]:
for sample, missing_ranges in sample_missing_regions.items():    
    for gmut in ordered_gmuts:
        gene, aapos = split_mut(gmut)
        start, end = get_gene_nt_coords(gff, gene, aapos)
        for missing_start, missing_end in missing_ranges:
            if missing_end >= start >= missing_start or missing_end >= end >= missing_start:
                print(gmut)
                print(start, end)
                print(missing_start, missing_end)
                print('='*80)
                dfaa_sub.loc[sample, gmut] = '*No Coverage'

ORF1a:T708I
2387 2390
2316 2418
ORF1a:A1204V
3875 3878
3770 4018
ORF1a:2037:MRAS
6374 6377
6248 6818
ORF1a:N2038D
6377 6380
6248 6818
ORF1a:T2093I
6542 6545
6248 6818
ORF1a:S2255F
7028 7031
7002 7084
S:V143-
21989 21992
21905 22061
S:Y144-
21992 21995
21905 22061
S:Y145D
21995 21998
21905 22061
S:L1265I
25355 25358
25083 25706
ORF3a:S40L
25510 25513
25083 25706
ORF3a:Q57H
25561 25564
25083 25706
ORF3a:V90F
25660 25663
25083 25706
ORF3a:H93Y
25669 25672
25083 25706
ORF1a:C2210F
6893 6896
6867 6904
ORF1a:S2255F
7028 7031
7021 7036
S:T22I
21626 21629
21624 21667
S:F486L
23018 23021
22999 23046
ORF1a:D4200Y
12863 12866
12846 12877
ORF3a:L219V
26047 26050
25918 26058
ORF3a:T221S
26053 26056
25918 26058
ORF1a:N2038D
6377 6380
6378 6429
ORF1a:C2210F
6893 6896
6860 6917
ORF1a:V2453I
7622 7625
7524 7645
ORF1a:S2255F
7028 7031
6898 7244
ORF1a:L2304S
7175 7178
6898 7244
S:V143-
21989 21992
21911 22209
S:Y144-
21992 21995
21911 22209
S:Y145D
21995 21998
21911 22209
ORF1a:S2255F
7028 7031
6964 7244

In [136]:
dfaa_sub

,ORF1a:T265I,ORF1a:T708I,ORF1a:A1204V,ORF1a:A1283V,ORF1a:A1314V,ORF1a:2037:MRAS,ORF1a:N2038D,ORF1a:T2093I,ORF1a:C2210F,ORF1a:S2255F,...,E:L19F,E:P71S,ORF8:Q27*,ORF8:D35Y,ORF8:S43F,ORF8:E106*,N:A119S,N:P168S,N:S206P,N:T391I
sample,,,,,,,,,,,,,,,,,,,,,
4538,ORF1a,*No Coverage,*No Coverage,ORF1a,ORF1a,*No Coverage,*No Coverage,*No Coverage,ORF1a,*No Coverage,...,-,E,-,ORF8,ORF8,ORF8,-,N,N,N
4645,ORF1a,ORF1a,-,ORF1a,ORF1a,-,-,-,ORF1a,ORF1a,...,E,E,-,ORF8,ORF8,ORF8,-,N,N,N
4649,ORF1a,ORF1a,-,ORF1a,ORF1a,-,*No Coverage,-,*No Coverage,ORF1a,...,-,E,-,ORF8,-,ORF8,-,N,N,N
4534,ORF1a,ORF1a,-,ORF1a,ORF1a,ORF1a,ORF1a,ORF1a,ORF1a,ORF1a,...,-,E,-,ORF8,-,ORF8,-,N,N,N
4581,ORF1a,ORF1a,ORF1a,ORF1a,ORF1a,ORF1a,ORF1a,-,ORF1a,ORF1a,...,-,E,-,ORF8,-,ORF8,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USA/TX-HMH-MCoV-41555/2020,ORF1a,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
USA/TX-HMH-MCoV-8792/2020,ORF1a,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
USA/TX-HMH-MCoV-34510/2020,*No Coverage,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,*No Coverage,-,-


In [137]:
dfaa_sub[ordered_gmuts].to_csv('on-wtd-deer-aa-matrix.tsv', sep='\t')

In [138]:
dfaa_sub = dfaa_sub[ordered_gmuts]

In [139]:
mut_pattern_samples = defaultdict(list)
for sample, mut_pattern in dfaa_sub.apply(lambda x: '|'.join(x), axis=1).items():
    mut_pattern_samples[mut_pattern].append(sample)

Output basic table of sample names and number of collapsed taxa represented by a sample.

In [140]:
nr_samples = []
with open('aa-mut-pattern-collapsed-count.tsv', 'w') as fout:
    for samples in mut_pattern_samples.values():
        sample = samples[0]
        nr_samples.append(sample)
        fout.write(f'{sample}\t{len(samples) - 1}\n')

In [141]:
len(nr_samples)

37

In [142]:
dfaa_sub.loc[nr_samples,:].to_csv('on-wtd-deer-aa-matrix-collapsed-nr-samples.tsv', sep='\t')

In [143]:
from Bio import Phylo

In [144]:
subtree = Phylo.read('subtree-with-ref.mafft.iqtree.treefile', 'newick')

In [145]:
subtree.count_terminals()

158

In [146]:
name_node = {n.name: n for n in subtree.get_terminals()}

In [147]:
nr_samples = set(nr_samples)

In [148]:
for name, node in name_node.items():
    if name in nr_samples:
        continue
    subtree.prune(node)

In [149]:
subtree.count_terminals()

37

Output tree with collapsed taxa

In [150]:
Phylo.write(subtree, 'subtree-collapsed.newick', 'newick', format_branch_length="%f")

1